# Курсовая работа

Импорт библиотек

In [1]:
import re, math, codecs, os

In [2]:
from pprint import pprint
import numpy as np

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models import Word2Vec

Подключение модели

In [227]:
m = "to.bin"

In [228]:
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True, unicode_errors='ignore')

Считывание предложений из файла

In [5]:
def cos(vA, vB):
    cos = np.dot(vA, vB) / (np.sqrt(np.dot(vA,vA)) * np.sqrt(np.dot(vB,vB)))
    return cos

In [167]:
info = []
words = []
final = []
with open('sentences.txt') as fh:
    sentences = fh.readlines()
for sentence in sentences:
    sentence.strip()
    sentence = sentence.split()
    count = 0
    for word in sentence:
        info.append(word)
        if len(info) == 3:
            words.append(info)
            info = []
    final.append(words)
    words = []

## Подсчет близости слов справа и слева

In [168]:
similar_words = []
for sentence in final:
    left_list = []
    best = -100
    ind = 0
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1 # Индикатор было ли то есть
            continue
        if not ind:
            pass
            try:
                left_list.append(model.wv[word[1]]) # Собираем список векторов левого контекста
            except Exception as e:
                print(e)
        elif ind:
            try:
                right = model.wv[word[1]]
                for vector in left_list:
                    best = max(best, cos(vector, right)) # Максимальное значение косинусной близости пар слов
            except Exception as e:
                continue
#                print(e)
    similar_words.append(best)

"word 'летесть' not in vocabulary"
"word 'летесть' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'усмановна' not in vocabulary"
"word 'усмановна' not in vocabulary"
"word 'шурупский' not in vocabulary"
"word 'шурупский' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'тераэлектрон' not in vocabulary"
"word 'тераэлектрон' not in vocabulary"
"word 'клишаса' not in vocabulary"
"word 'брейвик' not in vocabulary"
"word 'брейвик' not in vocabulary"
"word 'сабетта' n

## Кусочек кода для подсчета близости контекстов справа и слева

In [169]:
sentence_counts = []
for sentence in final:
    left_sum = 0
    right_sum = 0
    count_l = 0
    count_r = 0
    ind = 0
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1 # Индикатор было ли то есть
            continue
        if not ind:
            pass
            try:
                count_l += 1
                left_sum += model.wv[word[1]] # Складываем слова слева
            except Exception as e:
                print(e)
        elif ind:
            try:
                count_r += 1
                right_sum += model.wv[word[1]] # Складываем слова справа
            except Exception as e:
                continue
#                print(e)
    if count_l != 0 and count_r != 0:
        count = (left_sum / count_l, right_sum / count_l)
        sentence_counts.append(count)
    else:
        count = (left_sum, right_sum)
        sentence_counts.append(count)        

"word 'летесть' not in vocabulary"
"word 'летесть' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'эсэмэску' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'биоресурсы' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'амфимиксис' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'двухипостасный' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'лазебной' not in vocabulary"
"word 'усмановна' not in vocabulary"
"word 'усмановна' not in vocabulary"
"word 'шурупский' not in vocabulary"
"word 'шурупский' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'трлн' not in vocabulary"
"word 'тераэлектрон' not in vocabulary"
"word 'тераэлектрон' not in vocabulary"
"word 'клишаса' not in vocabulary"
"word 'брейвик' not in vocabulary"
"word 'брейвик' not in vocabulary"
"word 'сабетта' n

In [170]:
count = 1
sim = []
for i in sentence_counts:
    sim.append([count, cos(i[0], i[1])])
    count += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [171]:
# Есть случаи, когда один из контекстов не определен
for i in sim:
    if isinstance(i[1], np.ndarray):
        i[1] = 0

In [172]:
print(len(sim))
sorted_sim = sorted(sim, key=lambda x: x[1], reverse=True)

1000


In [173]:
# Запишем все в файл, чтобы работать с ним на локальном компьютере
with open('part_sim.txt', 'w') as fw:
    for i in sorted_sim:
        a, b = i
        fw.write(str(a) + ' ' + str(b) + '\n')

## Средние векторы по каждому типу значения

In [174]:
# Вектор - сумма векторов слов
context = [sum(i) for i in sentence_counts]

In [175]:
# Скачиваем разметку
with open('Разметка.txt') as fh:
    tags = list(map(lambda l: l.strip(), fh.readlines()))
print(len(tags))

1000


In [176]:
from collections import Counter

In [177]:
unique = Counter(tags)
unique

Counter({'дискурс': 12,
         'дополнение': 83,
         'интерпретация': 234,
         'коррекция': 40,
         'модальность': 2,
         'обобщение': 51,
         'обр определение': 4,
         'определение': 81,
         'переспрос': 7,
         'переспрос инт': 49,
         'переспрос как': 7,
         'переформулирование': 109,
         'пример': 8,
         'следствие': 258,
         'спецификация': 55})

In [178]:
# Находим средние вектора
average = {}
for x, y in zip(tags, context):
    if x not in average.keys():
        average[x] = 0
    average[x] += y
for name, count in unique.items():
    average[name] /= count    

In [179]:
from pymystem3 import Mystem
m = Mystem()

In [180]:
# Это функция для парсинга предложения
def vectorise(item):
    item = re.sub('[Тт]о есть', 'тоесть', item)
    punctuation = (list(""",;:”$%^&*№()_—=+!...?|[]{}\"/<>`~±§«»°1234567890.""")
               + ['- ', ' -', ' - ', '― ', ' ―', ' ― ', " '", "' "])
    item = item.lower()
    for char in punctuation:
        item = item.replace(char, ' ')
    item = item.strip()
    lemmas = m.lemmatize(item)
    vector = 0
    for word in lemmas:
        if word != 'тоедать' and ' ' not in word:
            try:
                vector+=model.wv[word]
            except Exception as e:
                pass
    return vector

In [181]:
# Это функция для подсчета близости к типам значения
def find_type(sentence):
    vector = vectorise(sentence) # Превращаем предложение в вектор
    types = []
    for key in average.keys():
        types.append((key, cos(average[key], vector))) # Считаем близость нашего вектора и вектора-прототипа
    types = sorted(types, reverse=True, key=lambda x: float(x[1])) # Сортируем
    return types

## Оценка предсказательной силы

In [182]:
with open('examples.txt') as fh:
    lines = fh.readlines()

In [183]:
pprint(lines[:5])

['%дискурс\n',
 'А вот о возможностях фото и видео контента стоит рассказать немного '
 'подробнее. </s> <s> То есть в том понимании, как видит все это наша студия и '
 'ее сотрудники. !\n',
 'В этот день Якопо Бельбо глядел в глаза Истине … И это, то есть мгновение, '
 'когда он затормозил пространство и время !\n',
 'Раз предлагал даже зайти в ближайший салон связи, купить ей телефон и только '
 'оставить свой - то есть перезвонила бы если захотела .. !\n',
 '―\xa0Нам\xa0нужно,\xa0чтоб\xa0нас\xa0всех\xa0черти\xa0разорвали… То есть, '
 'черт возьми, я говорю, чтоб, тысяча чертей, тысяча залпов вам в задницу… '
 'черт!  [Митьки. Папуас из Гондураса (1987)] [омонимия не снята] !\n']


In [184]:
meanings = {}
for line in lines:
    if line.startswith('%'): # В файле идет название значения, а потом примеры на него
        line = line.strip()
        line = line.strip('%')
        key = line
        meanings[key] = []
    else:
        line = line.strip()
        line = line.replace(u'\xa0', ' ')
        line = re.sub('\[.+?\]', '', line)
        line = re.sub('\<.+?\>', '', line)
        meanings[key].append(line)

In [185]:
meanings['дискурс']

['А вот о возможностях фото и видео контента стоит рассказать немного подробнее.   То есть в том понимании, как видит все это наша студия и ее сотрудники. !',
 'В этот день Якопо Бельбо глядел в глаза Истине … И это, то есть мгновение, когда он затормозил пространство и время !',
 'Раз предлагал даже зайти в ближайший салон связи, купить ей телефон и только оставить свой - то есть перезвонила бы если захотела .. !',
 '― Нам нужно, чтоб нас всех черти разорвали… То есть, черт возьми, я говорю, чтоб, тысяча чертей, тысяча залпов вам в задницу… черт!    !',
 'Правой рукой он обнял, тоже распотевшего ираскрасневшегося, кума Селивана и о чем-то горячо рассуждал с ним. Сердце у бабы замерло: «запил» мелькнуло у нее в голове, и ноги ее подкосились.  ― То есть… я тебе говорю… во всякое время… и больше ничего…   ',
 '― И не «сливайтесь»!  То есть, я вам скажу!..   Он ухватился обеими руками за голову. !',
 'Для вас.. для маменьки вашей…  То есть, господи, боже мой!   И снова начиналось целовани

In [186]:
median_prediction = []
for key, sentences in meanings.items():
    position = []
    for sentence in sentences:
        types = find_type(sentence) # Ищем ранжирование типов по каждому предложению
        for i in range(1, len(types)+1):
            if types[i-1][0] == key: # Запоминаем позицию нужного
                position.append(i)
    median_prediction.append((key, np.median(position))) # Собираем медианы предсказаний

Получается, что лучше всего программа ловит дискурсивные значения *то есть*, а не союзные

In [187]:
median_prediction = sorted(median_prediction, key=lambda x: x[1])
median_prediction

[('коррекция', 1.0),
 ('определение', 2.0),
 ('переформулирование', 3.0),
 ('переспрос', 3.0),
 ('следствие', 3.0),
 ('переспрос инт', 3.0),
 ('дискурс', 4.0),
 ('дополнение', 4.5),
 ('интерпретация', 6.0),
 ('пример', 7.0),
 ('спецификация', 9.0),
 ('обобщение', 9.5),
 ('переспрос как', 14.0)]

## Попробуем сравнить ожидания и реальность...

In [188]:
sentence = 'И здесь соблюдалась полная демократия, то есть картину отбирал по своему вкусу очередной дежурный '

Это был пример следствия

In [189]:
find_type(sentence)

[('переформулирование', 0.49505597),
 ('дополнение', 0.47478834),
 ('переспрос инт', 0.4563621),
 ('следствие', 0.45354623),
 ('спецификация', 0.4269766),
 ('интерпретация', 0.41820863),
 ('пример', 0.39940092),
 ('коррекция', 0.3921133),
 ('определение', 0.38180098),
 ('обр определение', 0.3462822),
 ('обобщение', 0.29192683),
 ('дискурс', 0.27698934),
 ('модальность', 0.23924688),
 ('переспрос', 0.22263747),
 ('переспрос как', 0.07655897)]

In [190]:
sentence = 'Очень, очень, очень желает с тобой познакомиться! — Да с какой же стати очень-то? — То есть не то чтобы... видишь, в последнее время, вот как ты заболел, мне часто и много приходилось об тебе поминать... Ну, он слушал... и как узнал, что ты по юридическому и кончить курса не можешь, по обстоятельствам, то сказал: «Как жаль!» Я и заключил... то есть всё это вместе, не одно ведь это; вчера Заметов...'

А вот тут да, дискурсивное + коррекция

In [191]:
find_type(sentence)

[('коррекция', 0.825968),
 ('модальность', 0.6326947),
 ('переспрос', 0.60248363),
 ('дискурс', 0.56397337),
 ('переспрос инт', 0.50616163),
 ('интерпретация', 0.43951437),
 ('следствие', 0.37636045),
 ('переформулирование', 0.3761081),
 ('дополнение', 0.35877144),
 ('спецификация', 0.32808265),
 ('пример', 0.2783088),
 ('обр определение', 0.22630312),
 ('переспрос как', 0.18950143),
 ('определение', 0.18448417),
 ('обобщение', 0.11021201)]

In [192]:
sentence = 'Не дальше, как сегодня, то есть в субботу, опять со статьей произошло что-то'

Дополнение

In [193]:
find_type(sentence)

[('коррекция', 0.5758741),
 ('модальность', 0.392787),
 ('переспрос', 0.34541526),
 ('дискурс', 0.28930727),
 ('переспрос инт', 0.27167675),
 ('интерпретация', 0.23072693),
 ('дополнение', 0.19555804),
 ('переформулирование', 0.1904719),
 ('следствие', 0.18356419),
 ('спецификация', 0.15842961),
 ('пример', 0.13711211),
 ('переспрос как', 0.12785417),
 ('определение', 0.05224668),
 ('обр определение', 0.047761932),
 ('обобщение', 0.013813176)]

In [194]:
sentence = 'Он вникал в глубину этого сравнения и разбирал, что такое другие и что он сам, в какой степени возможна и справедлива эта параллель и как тяжела обида, нанесенная ему Захаром; наконец, сознательно ли оскорбил его Захар, то есть убежден ли он был, что Илья Ильич всё равно, что «другой», или так это сорвалось у него с языка, без участия головы'

Интерпретация

In [195]:
find_type(sentence)

[('коррекция', 0.69024754),
 ('переспрос инт', 0.5751189),
 ('модальность', 0.5356691),
 ('переформулирование', 0.534296),
 ('дополнение', 0.5189285),
 ('интерпретация', 0.5130146),
 ('следствие', 0.4958651),
 ('переспрос', 0.49468702),
 ('спецификация', 0.45557415),
 ('дискурс', 0.4385344),
 ('пример', 0.4173915),
 ('определение', 0.3845572),
 ('обр определение', 0.36352557),
 ('обобщение', 0.2769479),
 ('переспрос как', 0.09013581)]

In [196]:
sentence = 'Недоверчив, скептик, циник... надувать любит, то есть не надувать, а дурачить...'

Коррекция

In [197]:
find_type(sentence)

[('коррекция', 0.60412174),
 ('переспрос', 0.4305266),
 ('дискурс', 0.41416016),
 ('модальность', 0.40667164),
 ('переспрос инт', 0.34752083),
 ('интерпретация', 0.30228963),
 ('переспрос как', 0.2653322),
 ('следствие', 0.25232676),
 ('спецификация', 0.23800765),
 ('переформулирование', 0.22608991),
 ('дополнение', 0.22078484),
 ('пример', 0.15159136),
 ('обр определение', 0.11262599),
 ('определение', 0.07879743),
 ('обобщение', 0.033312593)]

In [198]:
sentence = '― Я вечный двигатель изобрел.  ― То есть как? ― спросил Аркадий Максимович Фетисов. '

Переспрос как

In [199]:
find_type(sentence)

[('коррекция', 0.45573932),
 ('переспрос', 0.35592481),
 ('модальность', 0.30282316),
 ('дискурс', 0.29068756),
 ('переспрос как', 0.2662244),
 ('переспрос инт', 0.19788338),
 ('интерпретация', 0.15218261),
 ('дополнение', 0.14828637),
 ('спецификация', 0.1306065),
 ('переформулирование', 0.097923756),
 ('следствие', 0.0932395),
 ('пример', 0.043429617),
 ('обр определение', 0.026718834),
 ('определение', 0.00089571136),
 ('обобщение', -0.060099598)]

In [200]:
sentence = '― У меня было плохое настроение.   ― То есть?   ― Мне показалось, что все уже было… '

Переспрос

In [201]:
find_type(sentence)

[('коррекция', 0.80134976),
 ('модальность', 0.6743211),
 ('переспрос', 0.60172874),
 ('дискурс', 0.5097115),
 ('переспрос инт', 0.44786823),
 ('интерпретация', 0.41741678),
 ('следствие', 0.33139357),
 ('переформулирование', 0.2852703),
 ('дополнение', 0.2837346),
 ('спецификация', 0.26509923),
 ('переспрос как', 0.25157517),
 ('пример', 0.2274153),
 ('обр определение', 0.19354357),
 ('определение', 0.105108894),
 ('обобщение', 0.047165785)]

# Классификатор

Попробуем построить более сложный классификатор из встроенных в sklearn

In [202]:
import pandas as pd

In [203]:
# Создадим словарь объектов и ответов
dic = {}
dic['context'] = [i[1] for i in sim]
dic['type'] = tags

In [204]:
# Сделаем датафрейм
df = pd.DataFrame(dic)

In [205]:
X = df['context'].to_numpy().reshape(-1, 1) # Множество объектов
y = df['type'].to_numpy().reshape((-1, 1)) # Множество ответов

In [206]:
X[:2]

array([[0.36319342],
       [0.19858947]])

Создадим отложенную выборку для оценки качества модели

In [207]:
from sklearn.model_selection import train_test_split

In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Начнем работать с моделями

**KNN**

In [103]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [104]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [209]:
classifier = KNeighborsClassifier(n_neighbors=5, metric='cosine')
classifier.fit(list(X_train), y_train)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [210]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.268

In [144]:
from sklearn.metrics.pairwise import cosine_similarity

In [211]:
grid_params = {
    'n_neighbors' : [3, 5, 7],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan', 'cosine']
}

In [212]:
gs = GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    verbose=1,
    cv=3,
    n_jobs=-1
    )

In [213]:
gs_model = gs.fit(list(X_train), y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    5.1s finished
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

In [214]:
gs_model.best_params_

{'metric': 'cosine', 'n_neighbors': 7, 'weights': 'uniform'}

In [215]:
y_pred = gs_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.268

**Random Forest**

In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [216]:
rnd_clf = RandomForestClassifier(random_state=42)

In [217]:
rnd_clf.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [218]:
y_pred = rnd_clf.predict(X_test)

In [219]:
accuracy_score(y_test, y_pred)

0.188

In [220]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [221]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

rf_model = rf_random.fit(list(X_train), y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.3min finished
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:714: DataConversionWarning: A column-vector y was passed when a 1d array was expect

In [222]:
y_pred = rf_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.212

**LogReg**

In [223]:
from sklearn.linear_model import LogisticRegression

In [224]:
log_clf = LogisticRegression(random_state=42)

In [225]:
log_clf.fit(list(X_train), y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [226]:
y_pred = log_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.26